In [3]:
import pandas as pd

from collections import Counter as Ct

import numpy as np

import altair as alt
alt.data_transformers.enable("vegafusion") # for altair to work with large datasets

import calendar

In [4]:
# Open the Csv
directory = '' # Add the path

original_data = pd.read_csv(directory)

print(f"The Dataset has {len(original_data)} rows.")

The Dataset has 92865 rows.


In [3]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92865 entries, 0 to 92864
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   X                 90950 non-null  float64
 1   Y                 90950 non-null  float64
 2   OBJECTID          92865 non-null  int64  
 3   date_arr          92865 non-null  object 
 4   ArrestedAtAddr    92264 non-null  object 
 5   case_id           92462 non-null  float64
 6   ar_race           92710 non-null  object 
 7   ar_sex            92816 non-null  object 
 8   charge            92864 non-null  object 
 9   time_arr          92865 non-null  int64  
 10  city              92697 non-null  object 
 11  state             92784 non-null  object 
 12  zip               38620 non-null  object 
 13  typebond          83190 non-null  object 
 14  bond_amt          92865 non-null  float64
 15  district          90629 non-null  object 
 16  tract             90484 non-null  object

In [4]:
# Slicing the Data by columns to only subset columns that I will use for EDA
data = original_data[['OBJECTID', 'date_arr', 'ArrestedAtAddr', 'case_id', 
                      'ar_race', 'ar_sex', 'charge', 'time_arr', 'city', 'state']]

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92865 entries, 0 to 92864
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   OBJECTID        92865 non-null  int64  
 1   date_arr        92865 non-null  object 
 2   ArrestedAtAddr  92264 non-null  object 
 3   case_id         92462 non-null  float64
 4   ar_race         92710 non-null  object 
 5   ar_sex          92816 non-null  object 
 6   charge          92864 non-null  object 
 7   time_arr        92865 non-null  int64  
 8   city            92697 non-null  object 
 9   state           92784 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 7.1+ MB


In [6]:
# Checking missing values in each column

for col in list(data.columns):
    print(f"{len(data[data[col].isna()])} out of {len(data)} {col}(s) missing.\n")

0 out of 92865 OBJECTID(s) missing.

0 out of 92865 date_arr(s) missing.

601 out of 92865 ArrestedAtAddr(s) missing.

403 out of 92865 case_id(s) missing.

155 out of 92865 ar_race(s) missing.

49 out of 92865 ar_sex(s) missing.

1 out of 92865 charge(s) missing.

0 out of 92865 time_arr(s) missing.

168 out of 92865 city(s) missing.

81 out of 92865 state(s) missing.



In [7]:
#Replacing Missing Values with Keyword 'Missing' because most String Columns are missing Values
data = data.fillna('Missing')

In [8]:
# Now checking if there are any missing values
for col in list(data.columns):
    print(f"{len(data[data[col].isna()])} out of {len(data)} {col}(s) missing.\n")

0 out of 92865 OBJECTID(s) missing.

0 out of 92865 date_arr(s) missing.

0 out of 92865 ArrestedAtAddr(s) missing.

0 out of 92865 case_id(s) missing.

0 out of 92865 ar_race(s) missing.

0 out of 92865 ar_sex(s) missing.

0 out of 92865 charge(s) missing.

0 out of 92865 time_arr(s) missing.

0 out of 92865 city(s) missing.

0 out of 92865 state(s) missing.



In [9]:
# Duplicate Values
duplicates = data.duplicated()[data.duplicated() == True]
print(f'Number of Duplicates in this dataset is {len(duplicates)}')

Number of Duplicates in this dataset is 0


## Column Wise Data Check

In [10]:
# Checking for all the dates being in the Data range

data['date_arr'] = pd.to_datetime(data['date_arr'])

min_date = min(data['date_arr'])
max_date = max(data['date_arr'])

dateArrCheck = [0 if i >= min_date and i <= max_date else 1 for i in data['date_arr']]

# if any values outside of that range exist, this sum should be greater than 1

print(f'The amount of compromised Dates is: {sum(dateArrCheck)}')

The amount of compromised Dates is: 0


In [11]:
# Checking for case_id 


# Exlcuding missing values, changing the rest of ids into integer
cleaned_case_id = [int(i) for i in data['case_id'][data['case_id'] != 'Missing']]


min_id = min(cleaned_case_id)
max_id = max(cleaned_case_id)

print(min_id, max_id)

0 9999038314


In [12]:
# Testing case_id for being in its column limits

caseIDCheck = [0 if i >= min_id and i <= max_id else 1 for i in cleaned_case_id]

# if any values outside of that range exist, this sum should be greater than 1

print(f'The amount of compromised CaseIds is: {sum(caseIDCheck)}')

The amount of compromised CaseIds is: 0


In [13]:

nine_999 = [i for i in cleaned_case_id if str(i)[0:4] == '9999']

# Trying to find why these values have 9999 in the start of their case IDs
data.query('case_id in @nine_999')

,OBJECTID,date_arr,ArrestedAtAddr,case_id,ar_race,ar_sex,charge,time_arr,city,state
2016,2017,2011-10-28 04:00:00+00:00,Missing,9999038314.0,W,F,"NO OPERATOR LICENSE, M",1641,FAYETTEVILLE,NC
16812,16813,2015-03-29 04:00:00+00:00,2130 BRAGG BLVD/MCPHERSON AVE,9999008595.0,B,M,POSSESS CONTROL SUBSTANCE SCHEDULE VI,2258,FAYETTEVILLE,NC


In [14]:
counts = {'Less than 9': 0, 'Equal to 9': 0, 'Greater than 9': 0}
for i in cleaned_case_id:
    if len(str(i)) < 9:
        counts['Less than 9'] += 1
    elif len(str(i)) == 9:
        counts['Equal to 9'] += 1
    elif len(str(i)) > 9:
        counts['Greater than 9'] += 1
    else:
        name = f'Count {len(str(i))}'
        counts[name] = 0
        counts[name] += 1

print(counts)

{'Less than 9': 9, 'Equal to 9': 64, 'Greater than 9': 92389}


In [15]:
# How greater lengths are for the values longer than 9?
tens = []
for i in cleaned_case_id:
    if len(str(i)) == 10:
        tens.append(True)
    else:
        tens.append(False)

print(Ct(tens))
# This proves that All of the values greater than 9 are of length 10

Counter({True: 92389, False: 73})


In [16]:
less_than_9 = [] # for catching lengths of specifc ids
less_than_9_case_ids = [] # for catching specfic ids

for i in cleaned_case_id:
    if len(str(i)) < 9:
        less_than_9.append(len(str(i)))
        less_than_9_case_ids.append(i)

print(Ct(less_than_9))

#further investigating these short case_ids
data.query('case_id in @less_than_9_case_ids')

Counter({4: 3, 8: 3, 7: 2, 1: 1})


,OBJECTID,date_arr,ArrestedAtAddr,case_id,ar_race,ar_sex,charge,time_arr,city,state
1126,1127,2010-05-06 04:00:00+00:00,CLINTON RD,2010.0,I,M,LARCENY,2212,FAYETTEVILLE,NC
6273,6274,2012-12-19 05:00:00+00:00,6415 INDEPENDENCE PLACE DR,2192012.0,B,M,"BREAKING AND ENTERING - BUILDING, M",120,FAYETTEVILLE,NC
66066,66067,2018-05-01 04:00:00+00:00,410 WORTH ST,18011772.0,W,M,"FAILURE TO APPEAR, M",657,FAYETTEVILLE,NC
79818,473172,2021-01-13 05:00:00+00:00,6901 TWIN CREEK CT,2021.0,B,M,WARRANT/ORDER FOR ARREST,2116,FAYETTEVILLE,NC
80226,485744,2021-02-15 05:00:00+00:00,708 MAYFLOWER CT,2021.0,B,M,WARRANT/ORDER FOR ARREST,2335,FAYETTEVILLE,NC
84198,603785,2022-01-31 05:00:00+00:00,6900 CLIFFDALE RD,0.0,W,F,DRUGS- EQUIPMENT/PARAPHERNALIA - POSSESSING/C,1637,FAYETTEVILLE,NC
85180,632033,2022-04-15 04:00:00+00:00,3627 CANBY OAK LN,24983376.0,B,M,FAIL TO APPEAR,436,FAYETTEVILLE,NC
85678,647123,2022-05-21 04:00:00+00:00,1408 SKIBO RD,33052424.0,B,M,VANDALISM-TO REAL PROPERTY WILLFUL AND WANTON ...,125,FAYETTEVILLE,NC
92305,848276,2023-08-30 04:00:00+00:00,106 COUNTRY CLUB DR,2023021.0,B,M,TRESPASS (SECOND DEGREE),1512,FAYETTEVILLE,NC


In [17]:
# Checking Race data for inconsistencies
race_unique = data['ar_race'].unique()
race_unique

# Inconsistencies found need to fix the values with extra spaces
data['ar_race'] = [i.strip() for i in data['ar_race']]
print(data['ar_race'][data['ar_race'] != 'Missing'].unique()) #Rechecking to see if the values are corrected

['B' 'W' 'I' 'A' 'U' 'H' 'O' 'P']


In [18]:
# Checking Gender data for inconsistencies
print(data['ar_sex'][data['ar_sex'] != 'Missing'].unique())
# This data is consistent

['F' 'M' 'U']


In [19]:
# Checking the 'charge' column

unique_crimes = data['charge'].unique()

for i in unique_crimes:
    i # Add print() to print values, 

# It's hard to tell if any string has extra whitespaces the data also has multiple same valus with different looking 
data['charge'] = [i.strip() for i in data['charge']]

In [20]:
#Checking the 'time_arr' column

unique_time = data['time_arr'].unique()

min_time = min(data['time_arr'])
max_time = max(data['time_arr'])

timeArrCheck = [0 if i >= min_time and i <= max_time else 1 for i in data['time_arr']]

# if any values outside of that range exist, this sum should be greater than 1

print(f'The amount of compromised Times is: {sum(dateArrCheck)}')

The amount of compromised Times is: 0


In [21]:
# City data check
unique_city = data['city'].unique()
unique_city

#Understanding the patterns
for word in list(unique_city):
    print(word)

FAYETTEVILLE
Missing
LAURINBURG
LUMBER BRIDGE
FAYFETTEVILLE
FAY
SPRING LAKE
LUMBERTON
WOODLAND
FAYETTEVILLD
FAYETEVILLE
CLAYTON
RAEFORD
FAYETTEVILLE, NC
FAYETTEVILEL
FORT BRAGG
BLADEN COUNTY
PARKTON
COUNTY
RUTHERFORDTON
SANFORD
FAYETTEVILL
DUNN
FAYETTEVILE
HOPE MILLS
ST PAULS
EASTOVER
FAYTTEVILLE
DILLON
HM
HOKE
HOPE
ELIZABETHTOWN
WADESBORO
FT BRAGG
POPE AFB
OXFORD
FT. BRAGG
WOMACK
LILLINGTON
RED SPRINGS
GOLDSBORO
CHARLOTTE
CHAPEL HILL
CLINTON
RALEIGH
NEW HANOVER
FAYETTEVILLE        
MOORISVILLE, NC
MAXTON
ROSEBORO
ABERDEEN
SALEMBURG
CASTLE HAYNE
WILMINGTON
WHITAKERS
FAY 28301
FAY 28304
FAY 28314
FAY 28311
FAY 28303
FAY 28305
AUTRYVILLE
LINDEN
CNT
FAY 28306
HOPEMILLS
STEDMAN
WAGRAM
CONWAY
GREENSBORO NC
SHANNON NC
GODWIN
FAY 28312
TRENTON
FAYETTWEVILLE
WADE
BALTIMORE
NEWTON GROVE
FAYETTEVILLE`
ERWIN
PINEHURST
SPRINGLAKE
SAINT PAULS
BENSON
HARRELS
ROWLAND
FAAYETTEVILLE
GREENSBORO
FAIRMONT
MAGNOLIA
WINSTON SALEM
LAUREL HILLS
GARNER, NC
WILSON
PINEBLUFF
DUBLIN
BUNNLEVEL
ROCKINGHAM
TEACHEY
C

In [22]:
# Most values have same starting structure, that is 'FAY' but there is 'FAAY' as well which needs fixing

cleaned_fay = ['FAYETTEVILLE' if word.find('FAY') == 0 or word.find('FAAY') == 0 else word for word in list(data['city'])]

data['city'] = cleaned_fay

print(data['city'][data['city'] == 'FAYETTEVILLE'].unique()) # test to Checking if there are anymore wrong names for FAYETTEVILLE

['FAYETTEVILLE']


In [23]:
# state data

unique_states = data['state'].unique()
unique_states

# No inconsistencies found in states data

array(['NC', 'Missing', 'SC', 'AR'], dtype=object)

## Data Manipulation

In [24]:
# Making New columns Year, Month Num (Month Number) and Month Name

data['year'] = data['date_arr'].dt.year

data['month_num'] = data['date_arr'].dt.month

data['month_name'] = data['month_num'].apply(lambda x: calendar.month_name[x])

In [25]:
# Getting Data for FAYETTEVILLE only, because that's my focus of study

ville_Data = data.query('city == "FAYETTEVILLE" & state == "NC"')
ville_Data.head()

,OBJECTID,date_arr,ArrestedAtAddr,case_id,ar_race,ar_sex,charge,time_arr,city,state,year,month_num,month_name
0,1,2010-01-21 05:00:00+00:00,117 DRAKE STREET,2010002385.0,Missing,Missing,AUTO THEFT,1701,FAYETTEVILLE,NC,2010,1,January
1,2,2010-01-21 05:00:00+00:00,467 HAY STREET,2009041799.0,Missing,Missing,BREAKING OR ENTERING BUILDINGS GENERALLY,1040,FAYETTEVILLE,NC,2010,1,January
3,4,2010-09-18 04:00:00+00:00,BONNIE ST,2010031638.0,B,M,"CONTEMPT OF COURT / VIOLATING ORDERS OF COURT, M",1830,FAYETTEVILLE,NC,2010,9,September
4,5,2010-07-30 18:58:00+00:00,429 GILLESPIE ST,2010025041.0,B,M,"ASSAULT INFLICTING SERIOUS INJURY, M",1458,FAYETTEVILLE,NC,2010,7,July
5,6,2010-01-20 05:00:00+00:00,412 RUSSELL STREET,2009042883.0,B,M,OBTAIN MONEY FALSELY REPRESENTING PHYSICAL DEFECT,1533,FAYETTEVILLE,NC,2010,1,January


In [26]:

# State should be NC because FAYETTEVILLE is in North Carolina
print(ville_Data['state'].unique())

['NC']


In [27]:
# Selecting data from 2009 onwards for analysis, as the data before is very less
ville_Data = ville_Data.query('year > 2009')

###### I didn't change some columns to Integer because I won't use them for EDA. For example: case_id

--------------------------------------------------------------------

# Data Analysis

In [28]:
# Ville_Data Dataset
ville_Data.head()

,OBJECTID,date_arr,ArrestedAtAddr,case_id,ar_race,ar_sex,charge,time_arr,city,state,year,month_num,month_name
0,1,2010-01-21 05:00:00+00:00,117 DRAKE STREET,2010002385.0,Missing,Missing,AUTO THEFT,1701,FAYETTEVILLE,NC,2010,1,January
1,2,2010-01-21 05:00:00+00:00,467 HAY STREET,2009041799.0,Missing,Missing,BREAKING OR ENTERING BUILDINGS GENERALLY,1040,FAYETTEVILLE,NC,2010,1,January
3,4,2010-09-18 04:00:00+00:00,BONNIE ST,2010031638.0,B,M,"CONTEMPT OF COURT / VIOLATING ORDERS OF COURT, M",1830,FAYETTEVILLE,NC,2010,9,September
4,5,2010-07-30 18:58:00+00:00,429 GILLESPIE ST,2010025041.0,B,M,"ASSAULT INFLICTING SERIOUS INJURY, M",1458,FAYETTEVILLE,NC,2010,7,July
5,6,2010-01-20 05:00:00+00:00,412 RUSSELL STREET,2009042883.0,B,M,OBTAIN MONEY FALSELY REPRESENTING PHYSICAL DEFECT,1533,FAYETTEVILLE,NC,2010,1,January


In [29]:
# Grouping ville_Data by years

yearly_data = ville_Data.groupby('year')['OBJECTID'].count().reset_index().rename(columns = {'OBJECTID':'count'})
yearly_data['year'] = yearly_data['year'].apply(str)

In [30]:
# Grouping ville_Data by years and month to get monthly arrests data 

yearly_monthly_data = ville_Data.groupby(['year', 'month_num', 'month_name'])['OBJECTID']\
                    .count()\
                    .reset_index()\
                    .rename(columns = {'OBJECTID':'count'})\
                    .query('year > 2009')

yearly_monthly_data['year'] = yearly_monthly_data['year'].apply(str)


# Grouping ville_Data by months and averaging the counts to understand the overall monthly patterns for ALL TIME DATA

monthly_mean_data = yearly_monthly_data.groupby(['month_name', 'month_num'])['count'].apply(np.mean).reset_index().sort_values('month_num', ascending = True)
monthly_mean_data.rename(columns = {'count':'mean'}, inplace = True)

monthly_mean_data

,month_name,month_num,mean
4,January,1,557.714286
3,February,2,547.285714
7,March,3,581.071429
0,April,4,560.928571
8,May,5,569.785714
6,June,6,545.928571
5,July,7,575.928571
1,August,8,585.142857
11,September,9,556.071429
10,October,10,557.000000


In [31]:
# Month to Time Data

month_time = ville_Data[['month_num', 'month_name', 'time_arr']].sort_values('month_num', ascending = True)
month_time

,month_num,month_name,time_arr
0,1,January,1701
20331,1,January,2211
20335,1,January,156
20341,1,January,1440
58486,1,January,558
...,...,...,...
46048,12,December,1922
46041,12,December,2256
46027,12,December,558
45175,12,December,2159


In [32]:
# Getting time_arr from ville_Data and adding a new column for AM and PM times

overall_time_data = ville_Data['time_arr'].to_frame()
overall_time_data['time_status'] = ['AM' if i < 1200 else 'PM'for i in ville_Data['time_arr']]

In [33]:
### Ville_Data Min and Max MM,YYYY for titles of Charts

min_month_year = f"{calendar.month_name[min(ville_Data['date_arr']).month]}, {min(ville_Data['date_arr']).year}"
max_month_year = f"{calendar.month_name[max(ville_Data['date_arr']).month]}, {max(ville_Data['date_arr']).year}"


In [34]:
# yearly mean arrests
mean_yearly = round(np.mean(yearly_data['count']), 2)

In [35]:
# Overall gender data grouped 

overall_gender_data = ville_Data.groupby('ar_sex')['OBJECTID']\
                        .count()\
                        .reset_index()\
                        .rename(columns = {'OBJECTID':'count'})

overall_gender_data['percentage'] = [round(i / sum(overall_gender_data['count']) * 100, 2) for i in overall_gender_data['count']]

overall_gender_data = overall_gender_data.sort_values('percentage', ascending = False)

overall_gender_data

,ar_sex,count,percentage
1,M,65751,71.66
0,F,25488,27.78
3,U,466,0.51
2,Missing,46,0.05


In [36]:
# Overall race data grouped 

gender_race_data = ville_Data.groupby(['ar_sex', 'ar_race'])['OBJECTID'].count().reset_index().rename(columns = {'OBJECTID':'count'})

gender_race_data = gender_race_data.sort_values('count', ascending = False)
gender_race_data

,ar_sex,ar_race,count
9,M,B,46653
16,M,W,17285
1,F,B,15855
7,F,W,8752
11,M,I,1196
3,F,I,574
8,M,A,526
19,U,U,378
0,F,A,261
18,U,Missing,88


## Visuals

In [37]:
# Title Function  for plots

def chart_title_func(title, subtitle = False):
    if subtitle == False:
        return alt.Title(title, font = 'Century', 
                         fontSize = 18, 
                         orient = 'top', 
                         offset = 20)
    else:
        return alt.Title(title, font = 'Century', 
                         fontSize = 18, 
                         orient = 'top', 
                         offset = 20, 
                         subtitle = subtitle)

In [38]:
# Line Chart of yearly_data

line = alt.Chart(
    yearly_data, 
    title= chart_title_func('Count of Yearly Fayetteville Arrests', [f"Overall Yearly Arrest average from {min_month_year} to {max_month_year} : {mean_yearly}"]
    ))\
    .mark_line(color = '#ba3030',point=True)\
    .encode(
        x = alt.X('year', axis=alt.Axis(labelAngle=0)),
        y = 'count',
        tooltip = ['count']
    ).properties(
        width=900,
        height=300
    )

line

alt.Chart(...)

In [39]:
# Bar Chart for Arrests grouped by AM and PM

bar = alt.Chart(overall_time_data, title = chart_title_func('Arrests Counts by AM and PM'))\
    .mark_bar()\
    .encode(
        alt.X("time_status", axis=alt.Axis(labelAngle=0), title = ''),
        alt.Y('count()', title = 'Arrest Count'),
        tooltip=["count()"]
    ).properties(
        width=400,
        height=300
    )

bar

alt.Chart(...)

In [40]:
# Histogram of Arrest Time

color_scale = alt.Scale(
    type='linear',
    scheme='redyellowgreen',
    domainMid=0,
    reverse = True
)

hist_title = f"Arrest by hours"

hist = alt.Chart(overall_time_data, title = chart_title_func(hist_title, [f"This Histogram of All time data from {min_month_year} to {max_month_year}."]))\
    .mark_bar()\
    .encode(
        alt.X("time_arr:Q", bin=alt.Bin(maxbins=30), axis=alt.Axis(labelAngle=-45), title = 'Time'),
        alt.Y('count()', title = 'Arrest Count'),
        tooltip=["count()"],
        color = alt.Color('count():N', scale=color_scale)
    ).properties(
        width=900,
        height=250
    )


hist

alt.Chart(...)

In [41]:
# Bar of Arrest Time

bar_title = f"Arrests average for Each Month"

bar = alt.Chart(
    monthly_mean_data, 
    title = chart_title_func(bar_title, 
                             [f"This Chart is for {min_month_year} to {max_month_year}, for 2023 data is limited to September.", 
                             f"Average across all months is {round(np.mean(monthly_mean_data['mean']), 2)}"]))\
    .mark_bar()\
    .encode(
        alt.X("month_name", axis=alt.Axis(labelAngle=0), title = '').sort([i for i in monthly_mean_data['month_name']]),
        alt.Y('mean', title = 'Arrest Count Mean'),
        tooltip=["mean"]
    ).properties(
        width=900,
        height=250
    )


bar

alt.Chart(...)

In [42]:
# Overall Gender Data
overall_gender_data = overall_gender_data.sort_values('count', ascending = False)


gender_bar = alt.Chart(overall_gender_data, title = chart_title_func("Arrestee's Gender"))\
    .mark_bar(color = '#6291de')\
    .encode(
        alt.X("ar_sex", axis=alt.Axis(labelAngle=0), title = '').sort(['descending']),
        alt.Y('percentage', title = 'Percentage'),
        tooltip=["percentage"]
    ).properties(
        width=500,
        height=300
    )

gender_bar

alt.Chart(...)

In [44]:
# Gender Race Data

color_scale = alt.Scale(
    type='band',
    scheme = 'magma',
    domainMid= 0,
    reverse = False
)
    
race_gender_bar = alt.Chart(gender_race_data, title = chart_title_func("Arrestee's Race and Gender",))\
    .mark_bar()\
    .encode(
        alt.X("ar_race", axis=alt.Axis(labelAngle=0), title = 'Race').sort(['descending']),
        alt.Y('count', title = 'Arrest Count'),
        tooltip=["ar_sex", "count"],
        color = alt.Color('ar_sex', scale=color_scale).sort(['descending'])
    ).properties(
        width=800,
        height=300
    )

race_gender_bar

alt.Chart(...)